## 🧠 Clasificación por niveles de conectividad

Se parte de una variable continua llamada `Horas_Internet`, que representa las horas diarias de uso de internet por persona. A partir de ella, se crea una variable binaria llamada `Alta_conectividad`, que toma el valor `1` si una persona tiene **más de 3.5 horas de uso diario**, y `0` en caso contrario:

Esta línea genera una variable binaria (`0` o `1`) que puede utilizarse como variable objetivo (target) en un modelo de clasificación.

En lugar de limitarse a una clasificación binaria, es posible crear una **variable categórica ordinal** con más niveles. Por ejemplo:

- `0`: Baja conectividad (hasta 1 hora)
- `1`: Media conectividad (más de 1 hasta 3.5 horas)
- `2`: Alta conectividad (más de 3.5 horas)

Este enfoque permite reflejar con mayor fidelidad los distintos niveles de acceso a internet.

Ambos métodos son válidos. El primero entrega una variable numérica ordinal (útil para modelos), y el segundo produce una variable categórica con etiquetas descriptivas (ideal para visualización).

## 🤖 Regresión logística antes de limpieza

Se entrena un modelo básico de regresión logística para predecir la variable `Alta_conectividad` usando únicamente `Edad` e `Ingreso`, sin limpieza previa.

### Explicación línea por línea:
- `dropna()` elimina filas con valores faltantes.
- `train_test_split` divide los datos en entrenamiento y prueba (70/30).
- `LogisticRegression` se ajusta al conjunto de entrenamiento para predecir `Alta_conectividad`.
- `max_iter=200` asegura que el algoritmo tenga tiempo suficiente para converger.
- `accuracy_score` mide la proporción de predicciones correctas.
- El resultado final muestra la exactitud del modelo antes de cualquier limpieza de datos.

## 🧼 Limpieza de datos: Imputación, Codificación y Escalado

### Imputación con `SimpleImputer`

La imputación se usa para reemplazar los valores faltantes (`NaN`). Con `SimpleImputer` de `sklearn.impute`, se pueden aplicar distintas estrategias:

| Estrategia            | Qué hace                                            | Cuándo usarla                                      |
|------------------------|------------------------------------------------------|----------------------------------------------------|
| `'mean'`              | Sustituye con la media                               | Datos numéricos sin outliers                       |
| `'median'`            | Sustituye con la mediana                             | Datos numéricos con valores extremos o asimetría   |
| `'most_frequent'`     | Sustituye con el valor que más se repite (moda)      | Datos categóricos o discretos                      |
| `'constant'`          | Sustituye con un valor fijo definido por el usuario  | Para identificar o normalizar nulos explícitamente |

La elección de la estrategia depende del tipo de variable y la distribución de los datos. Por ejemplo, `Edad` suele tener valores atípicos, por lo tanto se imputa con la mediana. `Ingreso` se puede imputar con la media si su distribución no tiene sesgo fuerte.

También se puede aplicar `most_frequent` o `constant`:

### Codificación de variables categóricas

Los algoritmos de machine learning no trabajan directamente con texto, por lo que las variables categóricas deben transformarse a formato numérico. Se utiliza `pd.get_dummies()`.

Esto genera columnas binarias (0 o 1) por cada categoría, eliminando una de ellas (`drop_first=True`) para evitar colinealidad. Es esencial que todas las variables sean numéricas antes de pasar al modelo.

### Escalado de variables numéricas con `MinMaxScaler`

En modelos basados en distancias o coeficientes (KNN, SVM, regresión logística, redes neuronales), es crucial que las variables estén **en la misma escala**. Si no se escalan, una variable con valores grandes (como `Ingreso`) puede dominar el aprendizaje, incluso si no es más relevante.

`MinMaxScaler` transforma los valores de cada variable para que estén en el rango `[0, 1]`.

**Fórmula aplicada:**

\[ X_{\text{escalado}} = \frac{X - \min(X)}{\max(X) - \min(X)} \]

Esto garantiza que:

- El **mínimo** valor de la columna se convierte en `0`.
- El **máximo** valor se convierte en `1`.
- Los demás valores se reescalan proporcionalmente dentro de ese intervalo.

#### Ejemplo:

Si `Edad` tiene valores entre 18 y 90, y se escala con `MinMaxScaler`, entonces un valor de 54 se transforma en:
\[ \frac{54 - 18}{90 - 18} = \frac{36}{72} = 0.5 \]

#### Alternativas:

- `StandardScaler`: transforma los datos con media 0 y desviación estándar 1 (Z-score).
- `RobustScaler`: usa mediana e IQR, útil para datos con outliers.

## 🔁 Modelo después de la limpieza

Una vez que se han imputado, codificado y escalado los datos, se puede entrenar el modelo nuevamente y comparar el desempeño.

In [1]:
df_raw["Alta_conectividad"] = (df_raw["Horas_Internet"] > 3.5).astype(int)

NameError: name 'df_raw' is not defined

In [ ]:
def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw["Nivel_conectividad"] = df_raw["Horas_Internet"].apply(clasificar_conectividad)

In [ ]:
import pandas as pd

df_raw["Nivel_conectividad"] = pd.cut(
    df_raw["Horas_Internet"],
    bins=[-float("inf"), 1, 3.5, float("inf")],
    labels=["Baja", "Media", "Alta"]
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df_before = df_raw[["Edad", "Ingreso"]].dropna()
Xb = df_before
yb = df_raw.loc[df_before.index, "Alta_conectividad"]

Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.3, random_state=42)
model_before = LogisticRegression(max_iter=200)
model_before.fit(Xb_train, yb_train)
yb_pred = model_before.predict(Xb_test)
acc_before = accuracy_score(yb_test, yb_pred)
print("✅ Exactitud antes de la limpieza:", round(acc_before, 4))

In [ ]:
from sklearn.impute import SimpleImputer

df_clean = df_raw.copy()
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])

imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

In [ ]:
# Moda o constante (ejemplos)
SimpleImputer(strategy='most_frequent')
SimpleImputer(strategy='constant', fill_value='Desconocida')

In [ ]:
df_clean = pd.get_dummies(df_clean, columns=['Genero', 'Nivel_Educativo', 'Ciudad'], drop_first=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_clean[['Edad', 'Ingreso']] = scaler.fit_transform(df_clean[['Edad', 'Ingreso']])

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = StandardScaler()
# o
scaler = RobustScaler()
df_clean[['Edad', 'Ingreso']] = scaler.fit_transform(df_clean[['Edad', 'Ingreso']])

In [ ]:
Xc = df_clean.drop(columns=['Alta_conectividad'])
yc = df_clean['Alta_conectividad']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)

model_after = LogisticRegression(max_iter=200)
model_after.fit(Xc_train, yc_train)
yc_pred = model_after.predict(Xc_test)
acc_after = accuracy_score(yc_test, yc_pred)
print("✅ Exactitud después de la limpieza:", round(acc_after, 4))